In [1]:
import pandas as pd
from numpy import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import wandb

idle_time_data = pd.read_csv('../data/final_df_points_18_21_class.csv')

TargetVariable = ['idle_time']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'wind_speed', 'humidity', 'dt_start'
    , 'hex_enc', 'start_min', 'year', 'month', 'day', 'on_station', 'in_zone', 'zone_name_enc']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)

# my-awesome-swee

sweep_configuration_rfr = {
    "project": "RF-Regression",
    "name": "my-awesome-sweep-rfr",
    "metric": {"name": "r2_score", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "n_estimators": {
            "values": [8, 14, 19, 24, 32, 41, 54, 65, 78, 86, 98, 100]
        },
        "criterion": {
            "values": ['squared_error', 'poisson']
        },
        "max_depth": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, None]
        },
        "bootstrap": {
            "values": [True, False]
        },
        "max_features": {
            "values": ['auto', 'sqrt', 'log2']
        },
        "min_samples_leaf": {
            "values": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 18, 20, 25, 30]
        },
        "min_samples_split": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 18, 20, 25, 30]
        }
    }
}

In [2]:
def eval_regression(y_test,y_pred):
    # Metrics
    # r2, mae, mse, rmse
    r2 = r2_score(y_test, y_pred.ravel())
    mae = mean_absolute_error(y_test, y_pred.ravel())
    mse = mean_squared_error(y_test, y_pred.ravel())
    rmse = sqrt(mse)

    print('r2: %f' % r2)
    print('mae: %f' % mae)
    print('mse: %f' % mse)
    print('rmse: %f' % rmse)

    return r2, mse, rmse, mae


def my_train_func():
    wandb.init()

    _n_estimators = wandb.config.n_estimators
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _bootstrap = wandb.config.bootstrap
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = RandomForestRegressor(n_estimators=_n_estimators,
                                  criterion=_criterion,
                                  max_depth=_max_depth,
                                  bootstrap=_bootstrap,
                                  max_features=_max_features,
                                  min_samples_leaf=_min_samples_leaf,
                                  min_samples_split=_min_samples_split,
                                  n_jobs=2)

    model.fit(X_train, y_train.ravel())
    y_pred = model.predict(X_test)

    r2, mse, rmse, mae =eval_regression(y_test, y_pred)

    wandb.log({"r2_score": r2, "MSE": mse, "RMSE": rmse, "MAE" : mae})

In [ ]:
# wandb silent
import os
os.environ["WANDB_SILENT"] = "true"

# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration_rfr, project="RF-Regression")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func)

Create sweep with ID: lxq3f8ce
Sweep URL: https://wandb.ai/jonathanweske/RF-Regression/sweeps/lxq3f8ce
r2: 0.436576
mae: 210.561811
mse: 80884.892998
rmse: 284.402695

r2: 0.517612
mae: 189.086274
mse: 69251.457846
rmse: 263.156717

r2: 0.472152
mae: 197.536332
mse: 75777.678252
rmse: 275.277457

r2: 0.503681
mae: 193.172721
mse: 71251.276142
rmse: 266.929347

r2: 0.467471
mae: 203.855100
mse: 76449.640805
rmse: 276.495282

r2: 0.239270
mae: 255.193639
mse: 109210.038691
rmse: 330.469422

r2: 0.062928
mae: 284.143032
mse: 134525.670072
rmse: 366.777412

r2: 0.032073
mae: 287.378689
mse: 138955.126083
rmse: 372.766852

r2: 0.402503
mae: 225.827513
mse: 85776.365020
rmse: 292.876023

r2: 0.386128
mae: 229.009673
mse: 88127.173800
rmse: 296.862213

r2: 0.411236
mae: 221.261470
mse: 84522.650638
rmse: 290.727795

r2: 0.517059
mae: 188.077724
mse: 69330.792467
rmse: 263.307411

r2: 0.445423
mae: 209.144120
mse: 79614.795635
rmse: 282.160939

r2: 0.055076
mae: 285.720398
mse: 135652.790854
r